In [1]:
# importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Mounting the google drive

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# Loading the data in two Pandas dataframes

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/MRP/creditcard.csv')

In [4]:
# We split it into X_train, X_test, y_train, y_test for the purpose of classification_report and confusion_matrix.

from sklearn.model_selection import train_test_split

X = data.drop('Class', axis = 1).values
y = data['Class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=101)

In [5]:
# We split the data agian here in two dataframes; legitimate and anomaly. Then we drop the 'Class' column from out original 'data' 
# and take all the legitimate transactions in the data_legitimate variable and the anomalies in the data_anomaly variable.
# That is why this is unsupervised.

legitimate = data['Class'] == 0
anomaly = data['Class'] != 0

data.drop ('Class', axis = 1, inplace = True)
data_legitimate = data[legitimate]
data_anomaly = data[anomaly]

data_legitimate_train, data_legitimate_test = train_test_split(data_legitimate, test_size = 0.2, random_state=101)

print(f"Legitimate Count: {len(data_legitimate)}")
print(f"Anomaly Count: {len(data_anomaly)}")

print(f"Legitimate train Count: {len(data_legitimate_train)}")
print(f"Legitimate test Count: {len(data_legitimate_test)}")

Legitimate Count: 284315
Anomaly Count: 492
Legitimate train Count: 227452
Legitimate test Count: 56863


In [6]:
# importing the necessary libraries

import tensorflow as tf
from keras import models, layers
from keras.layers import Dense
from keras.models import Sequential

In [7]:
# defnining the autoencoder and compiling the model and fitting the model with data_legitimate_train only

model = Sequential()
model.add(Dense(50, input_dim = data_legitimate_train.shape[1], activation = 'relu'))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(data_legitimate_train.shape[1]))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(data_legitimate_train, data_legitimate_train, verbose = 1, epochs = 200)

Epoch 1/200
7108/7108 [==============================] - 13s 2ms/step - loss: 3048116.5000
Epoch 2/200
7108/7108 [==============================] - 13s 2ms/step - loss: 821.4500
Epoch 3/200
7108/7108 [==============================] - 13s 2ms/step - loss: 805.9020
Epoch 4/200
7108/7108 [==============================] - 13s 2ms/step - loss: 654.1246
Epoch 5/200
7108/7108 [==============================] - 13s 2ms/step - loss: 735.6958
Epoch 6/200
7108/7108 [==============================] - 13s 2ms/step - loss: 601.7866
Epoch 7/200
7108/7108 [==============================] - 13s 2ms/step - loss: 622.4993
Epoch 8/200
7108/7108 [==============================] - 13s 2ms/step - loss: 546.6263
Epoch 9/200
7108/7108 [==============================] - 14s 2ms/step - loss: 526.3131
Epoch 10/200
7108/7108 [==============================] - 13s 2ms/step - loss: 461.5175
Epoch 11/200
7108/7108 [==============================] - 13s 2ms/step - loss: 464.9218
Epoch 12/200
7108/7108 [=============

In [8]:
# The autoencoder model summary

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1550      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
_________________________________________________________________
dense_2 (Dense)              (None, 50)                300       
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1530      
Total params: 3,635
Trainable params: 3,635
Non-trainable params: 0
_________________________________________________________________


In [9]:
# The classification report for the autoencoder

from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict_classes(X_test)
predictions = np.where(predictions > 0, 1, 0)
print(classification_report(y_test, predictions))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56859
           1       0.00      0.00      0.00       103

    accuracy                           1.00     56962
   macro avg       0.50      0.50      0.50     56962
weighted avg       1.00      1.00      1.00     56962



In [10]:
# The consusion matrix for the autoencoder

confusion_matrix(y_test,predictions)

array([[56836,    23],
       [  103,     0]])

In [11]:
# The RMSE score for legitimate and anomaly section of the whole data

from sklearn import metrics

pred = model.predict(data_legitimate_test)
score1 = np.sqrt(metrics.mean_squared_error(pred, data_legitimate_test))
print(f"data_legitimate_test Score (RMSE): {score1}".format(score1))

pred = model.predict(data_legitimate)
score2 = np.sqrt(metrics.mean_squared_error(pred, data_legitimate))
print(f"data_legitimate Score (RMSE): {score2}")

pred = model.predict(data_anomaly)
score3 = np.sqrt(metrics.mean_squared_error(pred, data_anomaly))
print(f"anomaly Score (RMSE): {score3}".format(score1))

data_legitimate_test Score (RMSE): 2.0350027503631356
data_legitimate Score (RMSE): 2.1494482351845035
anomaly Score (RMSE): 5.3877557392120154
